# Guide Clustering Homework

I decided I wanted to try looking at this assignment like a classification or regression problem -- given features, predict a label (or a number).

But, the raw data doesn't consist of input features and output labels. Instead, our "input" is target DNA sequences and gene names, and our "output" is normalized cell counts with and without PLX after 0, 7, and 14 days, which translate to "activity" values.

First, I decided to take the input and engineer features from it, using several external libraries (OligoCalc, mfold, and Bowtie). You can see some analysis of those features (independently of their effect on activity) here: [Engineering gRNA Features](engineering-features.ipynb).

Then, I tried to use those engineered features to make predictions about activity. I tried doing regression to predict the actual activity value first, but it wasn't very accurate, so instead I tried converting the continuous activity values to discrete binary values (whether CRISPR "worked"), and the results were more interesting. You can see them here: [Binary Activity Classification](binary-classification.ipynb).

## Features

Check out [Engineering gRNA Features](engineering-features.ipynb) for plots.

### Raw sequence features

The simplest are features just based on the target DNA sequence / its RNA equivalent. These include `gc_content` and slightly more specific predicates like whether `sequence_starts_with_gg` or whether `sequence_contains_atg` (which I based on some of the suggestions in the assignment file).

### OligoCalc features

Probably the second-simplest are features based on the source code of [OligoCalc](http://biotools.nubic.northwestern.edu/OligoCalc.html), which is a simple javascript-based calculator for mass and thermodynamic quantities of single-stranded DNA and RNA. Its code is a bit messy, but examining it was a useful exercise. I reproduced their calculations for `molecular_mass`, as well as `nearest_neighbor_dS` (change in entropy) and `nearest_neighbor_dH` (change in enthalpy), which were in service of calculating `nearest_neighbor_Tm` (melting temperature). They were all effectively just weighted sums of counts of either individual or pairs of nucleotides.

### Mfold features

I had wanted to use OligoCalc for determining hairpins and other RNA secondary structure, but it seemed like something more fully-featured was required, so I downloaded [OligoArrayAux](http://unafold.rna.albany.edu/?q=DINAMelt/OligoArrayAux), which is a command line version of mfold / [Quikfold](http://unafold.rna.albany.edu/?q=DINAMelt/Quickfold). I used this to compute the most energetically favorable foldings of each gRNA sequence, which output both thermodynamic quantities (`mfold_dH`, `mfold_dS`, `mfold_dG`, and `mfold_Tm`) as well as hairpin details (which I featurized as `hairpin_stem_length`, `hairpin_loop_length`, `hairpin_start_index`, and `hairpin_count`).

Because running those command line tools is a bit computationally expensive, I precomputed them and saved them in a new CSV file. You can check out `guide/annotators.py`, `guide/mfold_wrapper.py` and `guide/mfold_result.py` for more details.

### Bowtie features

I used the same precomputation strategy for using [Bowtie](http://bowtie-bio.sourceforge.net/index.shtml) (version 1) to search for alignments (and close mismatches) for each target sequence, which I thought was important because the presence or absence of off-target matches is important when selecting gRNAs for use in CRISPR.

You can see more details about this in `guide/annotators.py`, `guide/bowtie_wrapper.py`, and `guide/bowtie_result.py`. I also validated that the Bowtie results I obtained made sense; you can see the analysis here: [Validating Bowtie Results](validating-bowtie-results.ipynb).

## Activity

See [Binary Activity Classification](binary-classification.ipynb) for the full analysis.

### Rationale for approach

I initially tried looking at the relationships between my gRNA features and the raw activity values of the data points, but it was hard to find any clear correlations. I even tried training a regressor to predict activity, but its accuracy was very low.

So I thought I might get more meaningful results if I simplified the problem; instead of solving a difficult regression problem, I could convert it into a simpler binary classification problem, where the boolean in question is "whether any genes were knocked out by CRISPR."

### Splitting the continuous distribution

For both the PLX and no-PLX cases, we can plot the activity distribution 3 ways: activity for the full two weeks, activity during just the first week, and activity during just the second week.

What jumped out at me was that the week-one and week-two results were very different.

In the base (no-PLX) case, there's a lot more activity in the first week than the second. This might be because CRISPR knocks out target genes relatively quickly, so if those target genes were essential, the affected cells' numbers will already be lowered after the first week. By the time the second week arrives, all of that has already happened, so any future activity might be more attributable to random (potentially almost-gaussian) noise than to CRISPR.

In the PLX case, it seems like the PLX isn't actually administered until the second week, considering the activity curves for the first week are almost identical with and without PLX. For week two, though, the curve shifts far into the positive, which is consistent with most of the cells being killed by the PLX. The ones that survive (have low or negative activities) likely do so because of CRISPR.

This suggests two strategies:
- Look at the points whose week-two PLX activity was negative (meaning more cells survived). These cells likely survived because of CRISPR. Consider them "active."
- Look at the points whose week-one base activity was very positive (meaning more cells died), _relative to the week-two activity distribution_. These cells likely died because of CRISPR. Consider them "active."

Then, for either strategy, we can consider the rest of the points (crucially, with some gap, to prevent any bleed-over from noise) to be "inactive." This inactive set will be much larger than the active set, but we can filter it down to just the points whose `target_gene` is the same as that of any of the points in the active set.

The rationale for this is that what we're interested in here is whether CRISPR "worked", i.e. whether a gene was knocked out. However, if a gene is nonessential to cell survival, its knockout might not affect gRNA activity, because nothing would happen either way. But if we assume the target genes of highly-active gRNAs are essential (because many cells died), we can infer that highly-inactive gRNAs targeting those same genes might be inactive because they can't be used in CRISPR, not because the knockouts they trigger have no effect on survivability.

This assumes, of course, that the highly-active gRNAs are only knocking out their target genes, which isn't always true because of off-target matches. But I didn't try to consider that in this analysis. I guess in the future I could restrict my initial split to very active gRNAs with few off-target matches.

Anyway, after applying this process, we obtain a smaller dataset with binary, rather than continuous activity values.

### Examining feature distributions for active vs. inactive points

After splitting the dataset, we can look at all our features and see if any of them have different distributions for active and inactive points.

I had to iterate a fair bit on the splitting process (which may have implicitly "overfit" my results), but interestingly, I found that I was able to obtain fairly different distributions for many of the features using the no-PLX splitting strategy but _not_ the PLX splitting strategy.

See the [unsuccessful PLX version](binary-classification-plx.ipynb) of the analysis, and compare it to the slightly more [successful no-PLX version](binary-classification.ipynb) which is kind of my main result.

### Training a classifier

This part ends up being kind of an afterthought -- once we have (relatively independent) features with different distributions for active and inactive points, we can train almost any classifier to distinguish between them with accuracy proportional to how little those feature distributions overlap. Based on how I split my data, I was able to get a random forest classifier to predict which group a point belonged to with consistent accuracy. For a rougher split (with more intermediate points where their activity was more ambiguous), I obtained accuracies between 60 and 65%. For a more well defined split, I was able to get that number up to around 72%.

## Conclusion

This was an interesting problem and I think I learned a lot about both CRISPR and data analysis by working on it! I don't know if any of these results or the code I used to generate them will be directly useful to you, but they were instructive for me to generate. Thanks for preparing this assignment, and I'm interested to hear how my approach compares to standards for predicting the outcomes of CRISPR experiments (and if you have any questions, thoughts, or feedback).